In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [2]:
CLIENT_ID = 'FDITTVSJQZVB3D0CHDYHK1INFTXOR0UYZ51XAKUDW1IGQLV5' # your Foursquare ID
CLIENT_SECRET = '2JT5IL5I0DGROK14M3PWMBG54MRW5QSKTO4DJCOKJRYAWKJ3' # your Foursquare Secret
ACCESS_TOKEN = '05B50VTMVA2NKP0V52WT5LL5IEKEQVXQVLA1BZSRMWSH0UOP' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FDITTVSJQZVB3D0CHDYHK1INFTXOR0UYZ51XAKUDW1IGQLV5
CLIENT_SECRET:2JT5IL5I0DGROK14M3PWMBG54MRW5QSKTO4DJCOKJRYAWKJ3


In [11]:
address = '13884 Metrotech Dr, Chantilly, VA 20151'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

38.89416166309194 -77.42442753740917


In [15]:
search_query = 'gym'
radius = 500
print(search_query + ' .... OK!')

gym .... OK!


In [16]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=FDITTVSJQZVB3D0CHDYHK1INFTXOR0UYZ51XAKUDW1IGQLV5&client_secret=2JT5IL5I0DGROK14M3PWMBG54MRW5QSKTO4DJCOKJRYAWKJ3&ll=38.89416166309194,-77.42442753740917&oauth_token=05B50VTMVA2NKP0V52WT5LL5IEKEQVXQVLA1BZSRMWSH0UOP&v=20180604&query=gym&radius=500&limit=30'

In [17]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '611526b78a557f2aaa5f8f51'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4c608276de6920a11e259664',
    'name': 'Chantilly Park Gym',
    'location': {'address': '3800 Lightfoot St',
     'lat': 38.89775085449219,
     'lng': -77.42633819580078,
     'labeledLatLngs': [{'label': 'entrance',
       'lat': 38.897676,
       'lng': -77.426326},
      {'label': 'display',
       'lat': 38.89775085449219,
       'lng': -77.42633819580078}],
     'distance': 432,
     'postalCode': '20151',
     'cc': 'US',
     'city': 'Chantilly',
     'state': 'VA',
     'country': 'United States',
     'formattedAddress': ['3800 Lightfoot St', 'Chantilly, VA 20151']},
    'categories': [{'id': '4bf58dd8d48988d176941735',
      'name': 'Gym',
      'pluralName': 'Gyms',
      'shortName': 'Gym',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
       'suffix': '.png'},
   

In [18]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

<ipython-input-18-5acf500bf9ad>:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  dataframe = json_normalize(venues)


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,4c608276de6920a11e259664,Chantilly Park Gym,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",v-1628776119,False,3800 Lightfoot St,38.897751,-77.426338,"[{'label': 'entrance', 'lat': 38.897676, 'lng'...",432,20151,US,Chantilly,VA,United States,"[3800 Lightfoot St, Chantilly, VA 20151]"


In [19]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Chantilly Park Gym,Gym,3800 Lightfoot St,38.897751,-77.426338,"[{'label': 'entrance', 'lat': 38.897676, 'lng'...",432,20151,US,Chantilly,VA,United States,"[3800 Lightfoot St, Chantilly, VA 20151]",4c608276de6920a11e259664


In [20]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel


In [21]:
# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Chantilly',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

In [22]:
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.6
    ).add_to(venues_map)


In [23]:
venues_map